In [1]:
#Importing Libraries


In [2]:
#Sentence Count


In [3]:
data['sentence_count'] = data['mda'].progress_apply(sentence_count)

100%|██████████| 126/126 [00:03<00:00, 40.81it/s]


In [4]:
#Word Count


In [5]:
data['word_count'] = data['mda'].progress_apply(word_count)

100%|██████████| 126/126 [00:12<00:00,  8.29it/s]


In [ ]:
#Show Head


In [6]:
#Syllable Count
import nltk
from nltk.corpus import cmudict
from nltk.tokenize import word_tokenize

#This variable d will store the dictionary CMU
d = cmudict.dict()

In [7]:
def complex_words(text):
    
    word_tokenize_list = word_tokenize(text)
    complexwords = 0
    
    #This funcition return the count of syllables from a word that exists in the dictionary
    def nsyl(word):
        return int([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0])
    
    for word in word_tokenize_list:
        syllables = 0
        
        if word.lower() in d:
            syllables = nsyl(word)
            
        #Count only word with more than 2 syllables
        if syllables >= 3:
            complexwords += 1
            
    return complexwords

In [8]:
data['complex_words'] = data['mda'].progress_apply(complex_words)

100%|██████████| 126/126 [00:18<00:00,  4.91it/s]


In [9]:
data['FOG_index'] = 0.4 * ((data['word_count']/data['sentence_count'])
                           + 100 * (data['complex_words'] / data['word_count']))

In [10]:
data.head()

,cik,company_name,date,industry_title,mda,sentence_count,word_count,complex_words,FOG_index
107,1101215,ALLIANCE DATA SYSTEMS CORP,2017-02-27,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSI\n...,496,15360,2917,19.983451
1094,1057352,COSTAR GROUP INC,2017-02-24,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...,387,13283,2735,21.965290
1283,1077561,"China Elite Information Co., Ltd.",2017-02-28,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...,24,751,188,22.529982
1510,1112985,Delta International Oil & Gas Inc.,2017-03-31,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...,108,3306,674,20.399314
1568,1065088,EBAY INC,2017-02-06,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...,356,13101,2538,22.469252


In [11]:
data.to_json('./data/mda.json')

## Dictionaries and Sentiment Analysis

In [2]:
neg_words = pd.read_fwf('./LoughranMcDonald_Negative.txt',header=None, 
                       names=['negatives'])

In [3]:
neg_words.head()

,negatives
0,ABANDON
1,ABANDONED
2,ABANDONING
3,ABANDONMENT
4,ABANDONMENTS


In [4]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [5]:
neg_stemmed = [stemmer.stem(word) for word in neg_words['negatives']]
neg_words['stemm'] = neg_stemmed

In [6]:
neg_words.head()

,negatives,stemm
0,ABANDON,abandon
1,ABANDONED,abandon
2,ABANDONING,abandon
3,ABANDONMENT,abandon
4,ABANDONMENTS,abandon


In [7]:
neg_word = set(list(neg_words['negatives']))
neg_stemmed = set(list(neg_words['stemm']))

In [8]:
from nltk.tokenize import word_tokenize
def negative_count (data, negative):    
    return len([word for word in word_tokenize(data) if word in negative])

In [9]:
negative_words = []
negative_stemmed_words = []

for mda in data['mda']:
    negative_words.append(negative_count(mda,negative=neg_word))
    
    stemmed_text = " ".join([stemmer.stem(word) for word in word_tokenize(mda)])
    negative_stemmed_words.append(negative_count(stemmed_text,negative=neg_stemmed))
    
data['negative_words'] = negative_words
data['negative_stemmed_words'] = negative_stemmed_words      

In [10]:
data.head()

,FOG_index,cik,company_name,complex_words,date,industry_title,mda,sentence_count,word_count,negative_words,negative_stemmed_words
107,19.983451,1101215,ALLIANCE DATA SYSTEMS CORP,2917,2017-02-27,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSI\n...,496,15360,125,205
1094,21.965290,1057352,COSTAR GROUP INC,2735,2017-02-24,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...,387,13283,130,209
1283,22.529982,1077561,"China Elite Information Co., Ltd.",188,2017-02-28,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...,24,751,6,14
1510,20.399314,1112985,Delta International Oil & Gas Inc.,674,2017-03-31,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...,108,3306,40,62
1568,22.469252,1065088,EBAY INC,2538,2017-02-06,"SERVICES-BUSINESS SERVICES, NEC",ITEM 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...,356,13101,128,203


In [11]:
#Saving the results
data.to_json('./data/mda.json')

### Stemming results

As we can see from the results above, the count of stemmed negative words often more than doubles the count of negative words. Does this indicate superior performance by the stemmed method? In the next lines of code we match the stemmed words to their original negative words. We find that the words content, subject, and liquid are a part of the stemmed negative word list. The stemmed negative word list has become too abstract and is now over estimating the number of negative words.

In [12]:
def negative_print (data, negative=neg_word):    
    return [word for word in word_tokenize(data) if word in negative]

negative_word_print = []
negative_stemmed_print = []

for mda in data['mda'].head(1):
    negative_word_print.append(negative_print(mda,negative=neg_word))
    
    stemmed_text = " ".join([stemmer.stem(word) for word in word_tokenize(mda)])
    negative_stemmed_print.append(negative_print(stemmed_text,negative=neg_stemmed))
    

In [13]:
print(set(negative_stemmed_print[0]))

{'limit', 'contract', 'declin', 'violat', 'subject', 'bankrupt', 'excess', 'critic', 'deterior', 'will', 'deceas', 'weak', 'liquid', 'unwil', 'bankruptci', 'breakag', 'delinqu', 'slow', 'concern', 'cancel', 'arrear', 'expos', 'defer', 'disclos', 'failur', 'late', 'loss', 'laps', 'correct', 'penalti', 'unabl', 'unpaid', 'uncollect', 'termin', 'close', 'downturn', 'deni', 'impair', 'bridg', 'content', 'neg', 'sever', 'object', 'fraud'}


In [14]:
neg_words[(neg_words.stemm == 'content') | (neg_words.stemm == 'subject')]

,negatives,stemm
341,CONTENTION,content
342,CONTENTIONS,content
2013,SUBJECTED,subject
2014,SUBJECTING,subject
2015,SUBJECTION,subject


## TF_IDF

One way to improve the results of the analysis above, or any dictionary-based or bag-of-words analysis, is to apply weights to the words based on their importance, or significance to the document in which they are found. Term frequency – inverse document frequency (TF-IDF) is a popular weighting algorithm for doing this. The idea behind TF-IDF is that terms thart are common in one document, but uncommon in the corpus, are important to describing the nature or topic of that one document. Terms that are common in every document, such as prepositions and determiners, are not as useful for describing the nature or topic of any document. Likewise, terms that appear infrequently in every document are also relatively unimportant. The formula for TF-IDF is as follows:

Implementing TF-IDF in code is fairly simple.

In [2]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

corpus = data['mda']
corpus_tfidf = tfidf_vectorizer.fit_transform(corpus)


print(corpus_tfidf[0,0:100])

  (0, 95)	0.00204082148842
  (0, 42)	0.00208370196469
  (0, 71)	0.00217878056615
  (0, 32)	0.00223196228408
  (0, 14)	0.00208370196469
  (0, 93)	0.00200054147252
  (0, 89)	0.00192664129441
  (0, 1)	0.00237241983961
  (0, 67)	0.00171824194977
  (0, 46)	0.00204082148842


## Cosine Similarity

A common task in text analytics is to measure document similarity, and one measure is Cosine Similarity. Cosine similarity takes as an input two same-length vectors and returns the cosine of the angle between those two vectors. If the angle is 0 degrees, then the vectors are identical and the cosine value is 1. If the angle is 90 degrees, then the vectors receive a cosine similarity score of zero.

In text analytics, vectors are constructed based on the count of words in two documents. Imagine we are evaluating the similarity of S1, S2 and S3:

&nbsp;&nbsp;&nbsp;S1: I read a book for a book club.

&nbsp;&nbsp;&nbsp;S2: I read a magazine.

&nbsp;&nbsp;&nbsp;S3: You read a pamphlet for a book club.


The vectors for these three strings can be constructed as follows:

||a|book|club|for|i|magazine|pamphlet|read|you|
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**S1**|2|2|1|1|1|0|i|1|0|
|**S2**|1|0|0|0|1|1|i|1|0|
|**S3**|2|1|1|1|0|0|i|1|1|


The cosine between two strings is: s = \sqrt{\frac{\sum_{i=1}^N (x_i - \overline{x})^2}{N-1} }, or

To improve the performance of this algorithm, for each string we can replace the raw counts of each word with the tf-idf weighted score. Tf-idf has two components, the frequency of each term, and the inverse document frequency of each term. The following table show the tf score for each word and for each document. The numerator is the number of times the word appears in the sentence. The denominator is the number of words in the sentence. Looking at the term frequency score, it is difficult to tell which word is the most important. 

||a|book|club|for|i|magazine|pamphlet|read|you|
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**S1**|2/8|2/8|1/8|1/8|1/8|0|0|1/8|0|
|**S2**|1/4|0|0|0|1/4|1/4|0|1/4|0|
|**S3**|2/8|1/8|1/8|1/8|0|0|1/8|1/8|1/8|

The inverse document frequency is a constant that applies to each word over the entire corpus. The idf(“a”) is the log of 3/3 because the word “a” appears in three out of three documents while idf(“book”) is log(3/1) because book appears in only one document. The next table shows the idf values for each word. Since idf is constant for each word across all documents in a corpus, the idf value does not change by document.

||a|book|club|for|i|magazine|pamphlet|read|you|
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**Sn**|log(3/3)|log(3/2)|log(3/2)|log(3/2)|log(3/2)|log(3/1)|log(3/1)|log(3/3)|log(3/1)|

To find the tf-idf weighted value of each word in each document, we now multiply the corresponding cells together and get the following values (rounded to two decimal places). For the first cell, the resulting value is 0 because log(3/3) = log(1) = 0. Thus, “a” is not an important word. 

||a|book|club|for|i|magazine|pamphlet|read|you|
|:---|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**S1**|0|0.044|0.022|.022|.022|0|0|0|0|
|**S2**|0|0|0|0|.044|0.119|0|0|0|
|**S3**|0|0.022|0.022|.022|0|0|0.060|0|0.060|

The table below shows the results of the Cosine Similarity algorithm comparing S1, S2, and S3. The top-right cells are the comparisons before the terms are weighted with tf-idf. The bottom left cells are the comparisons after the tf-idf weightings.

||S1|S2|S3|
|---|---|---|---|
|S1||0.577|0.822|
|S2|0.131||0.474|
|S3|0.358|0.000||

In [5]:
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np

In [6]:
#corpus = data['mda']
corpus_count = count_vectorizer.fit_transform(corpus)
corpus_tfidf = tfidf_vectorizer.fit_transform(corpus)

In [7]:
cosine_count = cosine_similarity(corpus_count)
cosine_tfidf = cosine_similarity(corpus_tfidf)

In [8]:
print(np.around(cosine_tfidf,4))

[[ 1.      0.765   0.6086 ...,  0.5167  0.7776  0.736 ]
 [ 0.765   1.      0.6876 ...,  0.5487  0.8912  0.8495]
 [ 0.6086  0.6876  1.     ...,  0.5034  0.6924  0.6438]
 ..., 
 [ 0.5167  0.5487  0.5034 ...,  1.      0.5715  0.5289]
 [ 0.7776  0.8912  0.6924 ...,  0.5715  1.      0.8725]
 [ 0.736   0.8495  0.6438 ...,  0.5289  0.8725  1.    ]]


In [9]:
print(np.around(cosine_count,4))

[[ 1.      0.9177  0.767  ...,  0.8608  0.9214  0.8846]
 [ 0.9177  1.      0.7858 ...,  0.8388  0.9626  0.9363]
 [ 0.767   0.7858  1.     ...,  0.789   0.774   0.7366]
 ..., 
 [ 0.8608  0.8388  0.789  ...,  1.      0.8525  0.8012]
 [ 0.9214  0.9626  0.774  ...,  0.8525  1.      0.9408]
 [ 0.8846  0.9363  0.7366 ...,  0.8012  0.9408  1.    ]]


In [10]:
for i in range(9):
    tfidf = np.argmax(cosine_tfidf[i,i+1:])
    count = np.argmax(cosine_count[i,i+1:])
    if tfidf != count:
        print('Document:', i, 'tf-idf:', tfidf, 'count:', count)

Document: 3 tf-idf: 7 count: 89
Document: 4 tf-idf: 28 count: 119
Document: 5 tf-idf: 118 count: 66
Document: 6 tf-idf: 117 count: 24
Document: 8 tf-idf: 42 count: 48


## Naïve Bayes Classification Algorithm

The Naïve Bayes classification algorithm is widely used as a baseline algorithm for machine learning performance. It is simple, intuitive, and easy to implement. Documents can be classified in any number of ways, but to evaluate performance of a classification algorithm, the outcome variable must be known beforehand. In this example, we are going to classify MD&A text according to industry. In other words, our classifier will determine which industry an MD&A comes from, and we will evaluate the results based on the actual industry association. 

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn import metrics

In [12]:
y = data['industry_title']
x_train_count, x_test_count, y_train_count, y_test_count = train_test_split(corpus_count, y, random_state=42)
x_train_tfidf, x_test_tfidf, y_train_ifidf, y_test_tfidf = train_test_split(corpus_count, y, random_state=42)

In [13]:
count_classifier = MultinomialNB()
tfidf_classifier = MultinomialNB()

In [ ]:
count_classifier.fit(x_train_count,y_train_count)
tfidf_classifier.fit(x_train_tfidf,y_train_ifidf)

In [17]:
y_predict_count = count_classifier.predict(x_test_count)
y_predict_tfidf = tfidf_classifier.predict(x_test_tfidf)

In [18]:
print(metrics.classification_report(y_test_count, y_predict_count))

print(metrics.classification_report(y_test_tfidf, y_predict_tfidf))

                                 precision    recall  f1-score   support

SERVICES-BUSINESS SERVICES, NEC       0.77      0.53      0.62        19
  SERVICES-PREPACKAGED SOFTWARE       0.53      0.77      0.62        13

                    avg / total       0.67      0.62      0.62        32

                                 precision    recall  f1-score   support

SERVICES-BUSINESS SERVICES, NEC       0.77      0.53      0.62        19
  SERVICES-PREPACKAGED SOFTWARE       0.53      0.77      0.62        13

                    avg / total       0.67      0.62      0.62        32



In [19]:
print(metrics.confusion_matrix(y_test_count, y_predict_count))

[[10  9]
 [ 3 10]]
